In [50]:
import pandas as pd
import numpy as np

In [2]:
import tarfile

In [13]:
tar1 = tarfile.open("../data/model_results.tar.gz")

In [21]:
df_rec = pd.read_csv(tar1.extractfile(tar1.getnames()[1]), header=0)

In [22]:
df_rec

,userId,movieId,prediction
0,4,318,4.624113
1,4,2571,4.556694
2,4,1196,4.533291
3,4,260,4.530176
4,4,58559,4.517324
...,...,...,...
99995,283185,166643,4.228540
99996,283185,3916,4.179170
99997,283185,8533,4.143071
99998,283185,160980,4.082581


In [23]:
tar2 = tarfile.open("../data/sample.tar.gz")

In [24]:
tar2.getnames()

['samples',
 'samples/sample_train.csv',
 'samples/sample.csv',
 'samples/sample_test.csv']

In [41]:
sample = pd.read_csv(tar2.extractfile(tar2.getnames()[2]),header=0)

In [42]:
sample['rank'] = sample[['userId','rating']].groupby("userId").rank(method='first').squeeze()

In [52]:
sample[["userId",'movieId','rank']]

,userId,movieId,rank
0,4,1,187.0
1,4,5,37.0
2,4,6,282.0
3,4,10,188.0
4,4,11,117.0
...,...,...,...
4340399,283185,4023,92.0
4340400,283185,4025,107.0
4340401,283185,4027,50.0
4340402,283185,4039,93.0


In [43]:
combine = pd.merge(left=sample,right=df_rec,on=["userId","movieId"],how='inner')

In [54]:
tt = combine[["userId",'rank']]
tt["rank"].apply(lambda x:1/np.log2(x+1))

0        0.122325
1        0.119341
2        0.119218
3        0.121243
4        0.154039
           ...   
55591    0.150844
55592    0.133195
55593    0.140267
55594    0.152925
55595    0.221065
Name: rank, Length: 55596, dtype: float64

# NDCG func

In [73]:
def NDCG(rec,test):
    """
    if ratings are equal, we choose dense rank.
    
    Args:
        rec(pd.DataFrame): the model recommended movies and predictions for users in testset
        sample(pd.DataFrame): the testset with user and ratings for movies
    Return:
        NDCG of this serie of recommendations on the users in test
    """
    test_copy = test.copy(deep=True)
    rec_copy = rec.copy(deep=True)
    k = rec_copy.groupby('userId')['movieId'].count().unique()
    if len(k) > 1:
        print("not all users have same number of recommendations!")
        return -1
    k = k[0]
    # generate rank for each users of movies in test
    test_copy['Rank'] = test_copy[['userId','rating']].groupby("userId").rank(method='dense').squeeze()
    test_copy = test_copy[["userId",'movieId','Rank']]
    # this would filter out any irrelavent movies using inner join with the testset
    combine = pd.merge(left=test_copy,right=rec_copy,on=["userId","movieId"],how='inner')
    combine = combine[['userId','Rank']]
    combine['Rank/log2'] = combine.Rank.apply(lambda x:1/np.log2(x+1))
    DCG = combine['Rank/log2'].sum() / combine.userId.nunique()
    IDCG = sum([1/np.log2(i+1) for i in range(1,k+1)])
    return DCG/IDCG

NDCG(df_rec,sample)

0.3463440476343716